<a href="https://colab.research.google.com/github/fathahmtk/CoreBalance-Accounting/blob/main/accounting_erp_app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from flask import Flask, render_template, request, redirect, url_for, send_file, session
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.sql import func
from functools import wraps
import io
from datetime import datetime

# Initialize Flask app
app = Flask(__name__)
# Configure SQLAlchemy to use SQLite database
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///erp.db'
# Disable SQLAlchemy modification tracking
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
# Set a secret key for session management
app.secret_key = 'secret' # In a production environment, use a strong, randomly generated key

# Initialize SQLAlchemy database
db = SQLAlchemy(app)

# --- Models ---
# Account model representing the chart of accounts
class Account(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    code = db.Column(db.String(10), unique=True, nullable=False)
    name = db.Column(db.String(100), nullable=False)
    type = db.Column(db.String(50), nullable=False) # e.g., Asset, Liability, Equity, Income, Expense

# JournalEntry model representing individual journal entries
class JournalEntry(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    date = db.Column(db.String(20), nullable=False) # Consider using Date type for better date operations
    description = db.Column(db.String(200), nullable=False)
    debit_account = db.Column(db.String(100), nullable=False) # Stores the name of the debit account
    credit_account = db.Column(db.String(100), nullable=False) # Stores the name of the credit account
    amount = db.Column(db.Float, nullable=False)

# User model for authentication
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(50), unique=True, nullable=False)
    password = db.Column(db.String(100), nullable=False) # In a production environment, hash passwords!

# --- Auth Decorator ---
# Decorator to protect routes that require login
def login_required(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        # Check if 'user_id' is in the session
        if 'user_id' not in session:
            # If not logged in, redirect to the login page
            return redirect(url_for('login'))
        # If logged in, proceed with the original function
        return f(*args, **kwargs)
    return decorated_function

# --- Login System ---
# Route for handling user login
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        # Get username and password from the form
        username = request.form['username']
        password = request.form['password']
        # Query the database for a matching user
        user = User.query.filter_by(username=username, password=password).first()
        # If user is found
        if user:
            # Store user id in the session
            session['user_id'] = user.id
            # Redirect to the home page
            return redirect(url_for('home'))
        # If login fails, you might want to add an error message here
    # Render the login page for GET requests or failed POST attempts
    return render_template('login.html')

# Route for handling user logout
@app.route('/logout')
def logout():
    # Remove user_id from the session
    session.pop('user_id', None)
    # Redirect to the login page
    return redirect(url_for('login'))

# --- Routes ---
# Home page route, requires login
@app.route('/')
@login_required
def home():
    return render_template('index.html')

# Accounts page route, requires login
@app.route('/accounts')
@login_required
def accounts():
    # Retrieve all accounts from the database
    all_accounts = Account.query.all()
    # Render the accounts template, passing the accounts data
    return render_template('accounts.html', accounts=all_accounts)

# Route for adding a new account, requires login
@app.route('/add_account', methods=['GET', 'POST'])
@login_required
def add_account():
    if request.method == 'POST':
        # Get account details from the form
        code = request.form['code']
        name = request.form['name']
        type = request.form['type']
        # Create a new Account object
        new_account = Account(code=code, name=name, type=type)
        # Add the new account to the database session
        db.session.add(new_account)
        # Commit the session to save the account to the database
        db.session.commit()
        # Redirect to the accounts page
        return redirect(url_for('accounts'))
    # Render the add account template for GET requests
    return render_template('add_account.html')

# Journal entries page route, requires login
@app.route('/journal_entries')
@login_required
def journal_entries():
    # Retrieve all journal entries from the database
    entries = JournalEntry.query.all()
    # Render the journal entries template, passing the entries data
    return render_template('journal_entries.html', entries=entries)

# Route for adding a new journal entry, requires login
@app.route('/add_entry', methods=['GET', 'POST'])
@login_required
def add_entry():
    if request.method == 'POST':
        # Get journal entry details from the form
        date = request.form['date']
        description = request.form['description']
        debit_account = request.form['debit_account']
        credit_account = request.form['credit_account']
        amount = float(request.form['amount'])
        # Create a new JournalEntry object
        new_entry = JournalEntry(date=date, description=description, debit_account=debit_account, credit_account=credit_account, amount=amount)
        # Add the new entry to the database session
        db.session.add(new_entry)
        # Commit the session to save the entry to the database
        db.session.commit()
        # Redirect to the journal entries page
        return redirect(url_for('journal_entries'))
    # Render the add entry template for GET requests
    return render_template('add_entry.html') # You'll need to populate dropdowns with account names here

# Ledger page route, requires login
@app.route('/ledger')
@login_required
def ledger():
    # Retrieve all accounts
    accounts = Account.query.all()
    ledger_data = []
    # Calculate ledger balance for each account
    for acc in accounts:
        # Sum of debits for the account
        debit_total = db.session.query(func.sum(JournalEntry.amount)).filter(JournalEntry.debit_account == acc.name).scalar() or 0
        # Sum of credits for the account
        credit_total = db.session.query(func.sum(JournalEntry.amount)).filter(JournalEntry.credit_account == acc.name).scalar() or 0
        # Calculate the balance
        balance = debit_total - credit_total
        ledger_data.append({
            'code': acc.code,
            'name': acc.name,
            'type': acc.type,
            'debit': debit_total,
            'credit': credit_total,
            'balance': balance
        })
    # Render the ledger template, passing the ledger data
    return render_template('ledger.html', ledger=ledger_data)

# Trial balance page route, requires login
@app.route('/trial_balance')
@login_required
def trial_balance():
    # Get start and end dates from query parameters
    start = request.args.get('start')
    end = request.args.get('end')
    query = JournalEntry.query
    # Filter journal entries by date range if provided
    if start and end:
        query = query.filter(JournalEntry.date >= start, JournalEntry.date <= end)

    # Retrieve all accounts
    accounts = Account.query.all()
    trial_data = []
    total_debit = 0
    total_credit = 0

    # Calculate trial balance for each account
    for acc in accounts:
        # Sum of debits for the account within the date range
        debit = query.with_entities(func.sum(JournalEntry.amount)).filter(JournalEntry.debit_account == acc.name).scalar() or 0
        # Sum of credits for the account within the date range
        credit = query.with_entities(func.sum(JournalEntry.credit_account)).filter(JournalEntry.credit_account == acc.name).scalar() or 0 # This line seems incorrect, should sum amount
        # Corrected line:
        credit = query.with_entities(func.sum(JournalEntry.amount)).filter(JournalEntry.credit_account == acc.name).scalar() or 0


        # Calculate the balance
        balance = debit - credit
        # Determine if the balance is a debit or credit balance
        debit_val = balance if balance > 0 else 0
        credit_val = abs(balance) if balance < 0 else 0

        # Accumulate total debits and credits
        total_debit += debit_val
        total_credit += credit_val

        trial_data.append({
            'code': acc.code,
            'name': acc.name,
            'debit': debit_val,
            'credit': credit_val
        })
    # Render the trial balance template, passing the trial balance data and totals
    return render_template('trial_balance.html', trial_data=trial_data, total_debit=total_debit, total_credit=total_credit, start=start, end=end)

# Profit and Loss statement route, requires login
@app.route('/profit_loss')
@login_required
def profit_loss():
    # Retrieve all accounts
    accounts = Account.query.all()
    income = 0
    expense = 0
    # Calculate total income and expense
    for acc in accounts:
        if acc.type == 'Income':
            # Sum of credits for income accounts
            income += db.session.query(func.sum(JournalEntry.amount)).filter(JournalEntry.credit_account == acc.name).scalar() or 0
        elif acc.type == 'Expense':
            # Sum of debits for expense accounts
            expense += db.session.query(func.sum(JournalEntry.amount)).filter(JournalEntry.debit_account == acc.name).scalar() or 0
    # Calculate profit or loss
    profit = income - expense
    # Render the profit and loss template
    return render_template('profit_loss.html', income=income, expense=expense, profit=profit)

# Bank reconciliation route, requires login
@app.route('/bank_reconciliation')
@login_required
def bank_reconciliation():
    # Retrieve all asset accounts
    accounts = Account.query.filter_by(type='Asset').all()
    # Retrieve journal entries related to asset accounts
    bank_entries = JournalEntry.query.filter(
        JournalEntry.debit_account.in_([a.name for a in accounts]) |
        JournalEntry.credit_account.in_([a.name for a in accounts])
    ).all()
    # Render the bank reconciliation template
    return render_template('bank_reconciliation.html', entries=bank_entries)

# Route to export trial balance as PDF, requires login
@app.route('/export_trial_balance')
@login_required
def export_trial_balance():
    # Import necessary libraries for PDF generation
    from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
    from reportlab.lib import colors
    from reportlab.lib.pagesizes import letter

    # Create a buffer to hold the PDF data
    buffer = io.BytesIO()
    # Create a new PDF document
    doc = SimpleDocTemplate(buffer, pagesize=letter)
    elements = []

    # Prepare data for the table
    data = [['Code', 'Account Name', 'Debit', 'Credit']]
    accounts = Account.query.all()
    for acc in accounts:
        # Calculate debit and credit totals for each account
        debit = db.session.query(func.sum(JournalEntry.amount)).filter(JournalEntry.debit_account == acc.name).scalar() or 0
        credit = db.session.query(func.sum(JournalEntry.amount)).filter(JournalEntry.credit_account == acc.name).scalar() or 0
        balance = debit - credit
        debit_val = balance if balance > 0 else 0
        credit_val = abs(balance) if balance < 0 else 0
        data.append([acc.code, acc.name, f"{debit_val:.2f}", f"{credit_val:.2f}"])

    # Create a table object
    table = Table(data)
    # Define table style
    table.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.grey), # Header background
        ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke), # Header text color
        ('ALIGN', (0,0), (-1,-1), 'CENTER'), # Center align all cells
        ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'), # Header font
        ('BOTTOMPADDING', (0,0), (-1,0), 12), # Header bottom padding
        ('BACKGROUND', (0,1), (-1,-1), colors.beige), # Data rows background
        ('GRID', (0,0), (-1,-1), 1, colors.black), # Add grid lines
    ]))
    # Add the table to the elements list
    elements.append(table)
    # Build the PDF document
    doc.build(elements)
    # Seek to the beginning of the buffer
    buffer.seek(0)
    # Send the PDF file as an attachment
    return send_file(buffer, as_attachment=True, download_name="trial_balance.pdf", mimetype='application/pdf')

# Main block to run the application
if __name__ == '__main__':
    # Create application context
    with app.app_context():
        # Create database tables if they don't exist
        db.create_all()
        # Add a default admin user if no users exist
        if not User.query.first():
            db.session.add(User(username='admin', password='admin')) # Remember to hash passwords in production!
            db.session.commit()
    # Run the Flask development server
    app.run(debug=True) # Set debug=False in production

ModuleNotFoundError: No module named 'flask_sqlalchemy'

In [1]:
!pip install Flask-SQLAlchemy